In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras
from sklearn.metrics import classification_report,confusion_matrix
import nltk
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'/content/Train_and_validation.csv')
df

,Unnamed: 0,Unnamed: 0.1,id,text,bias
0,0,0,1,Besides these forests naturally developed fore...,0
1,1,1,2,Your mother cleans the rice by separating husk...,0
2,2,2,3,All free gifts of nature and all things which ...,0
3,3,3,4,Humanism - the concept that man is the measure...,0
4,4,4,5,Various designs in clothing are found dependin...,0
...,...,...,...,...,...
1147,1991,1991,1992,"Mulberry , soyabean , cotton , tobacco , flax ...",1
1148,1993,1993,1994,b. Self Employment : Picture : . To eradicate ...,1
1149,1995,1995,1996,Bring both the ends of threads together and pu...,1
1150,1997,1997,1998,PotassiumProtein synthesisEnables the opening ...,1


In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

sw = stopwords.words('english')
unwanted = {"he", "him", "his", "himself", "she", "she's", "her", "hers", "herself", "they", "them", "their", "theirs", "themselves"}
sw = [el for el in sw if el not in unwanted]
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'it', "it's", 'its', 'itself', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 'v

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(sw)

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df.text = df.text.apply(clean_text)
df.text = df.text.str.replace('\d+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
import gensim
import gensim.downloader as gensim_api
nlp = gensim_api.load("fasttext-wiki-news-subwords-300")

[=================================================-] 99.9% 957.9/958.4MB downloaded

In [ ]:
corpus = df.text

## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

## detect bigrams and trigrams
bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, 
                 delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
            delimiter=" ".encode(), min_count=5, threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

In [ ]:
nlp = gensim.models.FastText(lst_corpus, size=300, window=8, min_count=1, sg=1, iter=30)

In [ ]:
  from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [ ]:
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=15, padding="post", truncating="post")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, df.bias, test_size = 0.1, random_state = 42) 

In [ ]:
# lamba = -1
# for el in range(len(X_train)):
#   if X_train[el][-1] != 0:
#     lamba = max(lamba, len(X_train[el]))

# lamba

In [ ]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
## code attention layer
def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

## input
x_in = layers.Input(shape=(15,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=15, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)
## final dense layers
x = layers.Dense(64, activation='relu')(x)
y_out = layers.Dense(1, activation='sigmoid')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 15, 300)      1487100     ['input_2[0][0]']                
                                                                                                  
 permute_1 (Permute)            (None, 300, 15)      0           ['embedding_1[0][0]']            
                                                                                                  
 dense_3 (Dense)                (None, 300, 15)      240         ['permute_1[0][0]']              
                                                                                            

In [ ]:
## encode y
dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(Y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in Y_train])
## train
training = model.fit(x=X_train, y=y_train, batch_size=32,
                     epochs=15, shuffle=True, verbose=1)
# y_train.shape

Epoch 1/15
33/33 [==============================] - 10s 35ms/step - loss: 0.6894 - accuracy: 0.5676
Epoch 2/15
33/33 [==============================] - 1s 41ms/step - loss: 0.6258 - accuracy: 0.6795
Epoch 3/15
33/33 [==============================] - 1s 40ms/step - loss: 0.5686 - accuracy: 0.7268
Epoch 4/15
33/33 [==============================] - 1s 45ms/step - loss: 0.5190 - accuracy: 0.7500
Epoch 5/15
33/33 [==============================] - 1s 44ms/step - loss: 0.4800 - accuracy: 0.7751
Epoch 6/15
33/33 [==============================] - 2s 47ms/step - loss: 0.4596 - accuracy: 0.7954
Epoch 7/15
33/33 [==============================] - 1s 45ms/step - loss: 0.4424 - accuracy: 0.7944
Epoch 8/15
33/33 [==============================] - 2s 47ms/step - loss: 0.4343 - accuracy: 0.8021
Epoch 9/15
33/33 [==============================] - 2s 45ms/step - loss: 0.4148 - accuracy: 0.8069
Epoch 10/15
33/33 [==============================] - 1s 44ms/step - loss: 0.4140 - accuracy: 0.8205
Epoch 11

In [ ]:
predicted_prob = model.evaluate(X_test, Y_test)
# predicted = [dic_y_mapping[np.argmax(pred)] for pred in
            #  predicted_prob]

4/4 [==============================] - 2s 21ms/step - loss: 0.3141 - accuracy: 0.8793


In [ ]:
predicted_prob = model.predict(X_test)
y_pred = list(map(lambda x: 1 if x>0.5 else 0,predicted_prob))
print(classification_report(Y_test,y_pred))


4/4 [==============================] - 2s 16ms/step
              precision    recall  f1-score   support

           0       0.92      0.81      0.86        54
           1       0.85      0.94      0.89        62

    accuracy                           0.88       116
   macro avg       0.88      0.88      0.88       116
weighted avg       0.88      0.88      0.88       116



In [ ]:
print(confusion_matrix(Y_test,y_pred))

[[44 10]
 [ 4 58]]


In [ ]:
X,y = X,df.bias

In [ ]:
X

array([[ 117,  508,  663, ...,   28,    0,    0],
       [  18, 2199,  509, ...,    0,    0,    0],
       [ 225,  510,  118, ...,    0,    0,    0],
       ...,
       [ 753, 1217, 2196, ...,    0,    0,    0],
       [4944, 4945, 1747, ..., 4952, 4953, 4954],
       [ 151,   36, 1709, ...,    0,    0,    0]], dtype=int32)

In [ ]:
predicted_prob = model.predict(X)
y_hat = list(map(lambda x: 1 if x>0.5 else 0,predicted_prob))
print(classification_report(Y_test,y_pred))


36/36 [==============================] - 1s 20ms/step
              precision    recall  f1-score   support

           0       0.92      0.81      0.86        54
           1       0.85      0.94      0.89        62

    accuracy                           0.88       116
   macro avg       0.88      0.88      0.88       116
weighted avg       0.88      0.88      0.88       116



In [ ]:
df["pred"] = y_hat
df


,Unnamed: 0,Unnamed: 0.1,id,text,bias,pred
0,0,0,1,besides forests naturally developed forests ma...,0,0
1,1,1,2,mother cleans rice separating husk mud particl...,0,0
2,2,2,3,free gifts nature things man cannot produce in...,0,0
3,3,3,4,humanism concept man measure things,0,0
4,4,4,5,various designs clothing found depending count...,0,0
...,...,...,...,...,...,...
1147,1991,1991,1992,mulberry soyabean cotton tobacco fla jinseng p...,1,1
1148,1993,1993,1994,b self employment picture eradicate widespread...,1,1
1149,1995,1995,1996,bring ends threads together put knot ends threads,1,0
1150,1997,1997,1998,potassiumprotein synthesisenables opening clos...,1,1


In [ ]:
text_df = df.drop(['Unnamed: 0',	'Unnamed: 0.1'],axis=1).set_index('id')
text_df

,text,bias,pred
id,,,
1,besides forests naturally developed forests ma...,0,0
2,mother cleans rice separating husk mud particl...,0,0
3,free gifts nature things man cannot produce in...,0,0
4,humanism concept man measure things,0,0
5,various designs clothing found depending count...,0,0
...,...,...,...
1992,mulberry soyabean cotton tobacco fla jinseng p...,1,1
1994,b self employment picture eradicate widespread...,1,1
1996,bring ends threads together put knot ends threads,1,0


In [ ]:
pair_df = pd.read_csv('/content/pairs.txt',names=['p1','p2','label'])
pair_df

,p1,p2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0
...,...,...,...
155946,546,1512,1
155947,720,1902,1
155948,1112,1591,0
155949,396,1472,1


In [ ]:
pred_label = []
for i in range(pair_df.shape[0]):
    pred_label.append(np.logical_xor(text_df.pred[pair_df.p1[i]],text_df.pred[pair_df.p2[i]]).astype(int))
#     break;

In [ ]:
pred_label

[0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,


In [ ]:
# from sklearn.metrics import classification_report
print(classification_report(pair_df.label.values,pred_label))
print(confusion_matrix(pair_df.label.values,pred_label))


              precision    recall  f1-score   support

           0       0.75      0.73      0.74     80400
           1       0.73      0.75      0.74     75551

    accuracy                           0.74    155951
   macro avg       0.74      0.74      0.74    155951
weighted avg       0.74      0.74      0.74    155951

[[59082 21318]
 [19222 56329]]


In [ ]:
len(text_df[text_df["bias"]!=text_df["pred"]]
)

172

In [ ]:
import pickle
  

# Open a file and use dump()
with open('fasttext_X.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(X, file)